# Training DRKG Using TransE_L2
This notebook shows how to train DRKG embeddings using TransE_L2

Before training the model, you need to download the original DRKG source file into your local storage, e.g., ./data/drkg.tsv

## Install DGL-KE
Before training the model, we need to install dgl and dgl-ke packages as well as other dependencies. 

In [4]:
!sudo pip3 install torch
!sudo pip3 install dgl 
!sudo pip3 install dglke

     |████████████████████████████████| 752.0MB 79.9MB/s eta 0:00:01
You should consider upgrading via the 'pip install --upgrade pip' command.
     |████████████████████████████████| 3.0MB 2.9MB/s eta 0:00:01
     |████████████████████████████████| 26.1MB 23.0MB/s eta 0:00:01
  Found existing installation: scipy 0.19.1
    Uninstalling scipy-0.19.1:
      Successfully uninstalled scipy-0.19.1
You should consider upgrading via the 'pip install --upgrade pip' command.
     |████████████████████████████████| 61kB 2.5MB/s eta 0:00:011
You should consider upgrading via the 'pip install --upgrade pip' command.


## Prepare train/valid/test set
Before training, we need to split the original drkg into train/valid/test set with a 9:0.5:0.5 manner.

In [20]:
import pandas as pd
import numpy as np
import sys
sys.path.insert(1, '../utils')
from utils import download_and_extract
download_and_extract()
drkg_file = '../data/drkg/drkg.tsv'

df = pd.read_csv(drkg_file, sep="\t")
triples = df.values.tolist()

We get 5,869,293 triples, now we will split them into three files

In [21]:
num_triples = len(triples)
num_triples
# Please make sure the output directory exist.
import os
#os.mkdir('train_Hetionet')

In [24]:
# Please make sure the output directory exist.
os.mkdir('train_DGIDB')
dataset_name='DGIDB'
with open("train_DGIDB/drkg_train.tsv", 'w+') as f:
    for idx in range(num_triples):
        if triples[idx][1].split(':')[0]==dataset_name:
            f.writelines("{}\t{}\t{}\n".format(triples[idx][0], triples[idx][1], triples[idx][2]))

## Training TransE_l2 model
We can training the TransE_l2 model by simplying using DGL-KE command line. For more information about using DGL-KE please refer to https://github.com/awslabs/dgl-ke.

Here we train the model using 8 GPUs on an AWS p3.16xlarge instance.

In [29]:
!DGLBACKEND=pytorch dglke_train --dataset DGIDB --data_path ./train_DGIDB --data_files drkg_train.tsv --format 'raw_udd_hrt' --model_name TransE_l2 --batch_size 2048 \
--neg_sample_size 256 --hidden_dim 400 --gamma 12.0 --lr 0.1 --max_step 100000 --log_interval 1000 --batch_size_eval 16 -adv --regularization_coef 1.00E-07  --num_thread 1 --gpu 0 1 2 3 4 5 6 7 --num_proc 8 --neg_sample_size_eval 10000 --async_update

Using backend: pytorch
Logs are being recorded at: ckpts/TransE_l2_DGIDB_4/train.log
Reading train triples....
Finished. Read 26290 train triples.
|Train|: 26290
random partition 26290 edges into 8 parts
part 0 has 3287 edges
part 1 has 3287 edges
part 2 has 3287 edges
part 3 has 3287 edges
part 4 has 3287 edges
part 5 has 3287 edges
part 6 has 3287 edges
part 7 has 3281 edges
/usr/local/lib/python3.6/dist-packages/dgl/base.py:25: UserWarning: multigraph will be deprecated.DGL will treat all graphs as multigraph in the future.
  warnings.warn(msg, warn_type)
Total initialize time 0.140 seconds
[proc 5][Train](1000/100000) average pos_loss: 0.3099977732151747
[proc 6][Train](1000/100000) average pos_loss: 0.2824301677383482
[proc 7][Train](1000/100000) average pos_loss: 0.30174483053386214
[proc 3][Train](1000/100000) average pos_loss: 0.3621343788704653
[proc 1][Train](1000/100000) average pos_loss: 0.23849174101650714
[proc 2][Train](1000/100000) average pos_loss: 0.25991080194558525


[proc 2][Train](4000/100000) average pos_loss: 0.18517213696241377
[proc 1][Train](4000/100000) average pos_loss: 0.1855486381351948
[proc 7][Train](4000/100000) average pos_loss: 0.18346590624749662
[proc 2][Train](4000/100000) average neg_loss: 0.3383186005204916
[proc 6][Train](4000/100000) average pos_loss: 0.1844317391961813
[proc 4][Train](4000/100000) average pos_loss: 0.18385540272295475
[proc 3][Train](4000/100000) average pos_loss: 0.18409426398575307
[proc 5][Train](4000/100000) average pos_loss: 0.1858911810964346
[proc 1][Train](4000/100000) average neg_loss: 0.3386798925101757
[proc 2][Train](4000/100000) average loss: 0.2617453686594963
[proc 7][Train](4000/100000) average neg_loss: 0.3376454005390406
[proc 6][Train](4000/100000) average neg_loss: 0.33730094639956953
[proc 4][Train](4000/100000) average neg_loss: 0.3389878822863102
[proc 0][Train](4000/100000) average pos_loss: 0.18483766327798368
[proc 3][Train](4000/100000) average neg_loss: 0.33466597375273704
[proc 1

[proc 3][Train](7000/100000) average pos_loss: 0.18383481860160827
[proc 1][Train](7000/100000) average pos_loss: 0.18549595932662488
[proc 0][Train](7000/100000) average pos_loss: 0.18599767872691156
[proc 3][Train](7000/100000) average neg_loss: 0.33664141938090325
[proc 6][Train](7000/100000) average pos_loss: 0.18619815094769002
[proc 1][Train](7000/100000) average neg_loss: 0.3397901753485203
[proc 7][Train](7000/100000) average pos_loss: 0.18361685082316398
[proc 2][Train](7000/100000) average pos_loss: 0.18415351328253746
[proc 0][Train](7000/100000) average neg_loss: 0.33958618661761286
[proc 3][Train](7000/100000) average loss: 0.2602381186336279
[proc 5][Train](7000/100000) average pos_loss: 0.18744936177134514
[proc 4][Train](7000/100000) average pos_loss: 0.18466899353265762
[proc 6][Train](7000/100000) average neg_loss: 0.33988662786781787
[proc 1][Train](7000/100000) average loss: 0.2626430678367615
[proc 7][Train](7000/100000) average neg_loss: 0.33828164665400984
[proc 

[proc 2][Train](10000/100000) average pos_loss: 0.18541604962944985
[proc 5][Train](10000/100000) average pos_loss: 0.18668844874203205
[proc 7][Train](10000/100000) average pos_loss: 0.18503255787491799
[proc 3][Train](10000/100000) average pos_loss: 0.18423546455800532
[proc 2][Train](10000/100000) average neg_loss: 0.3394985725283623
[proc 6][Train](10000/100000) average pos_loss: 0.1851583833694458
[proc 4][Train](10000/100000) average pos_loss: 0.18521355761587618
[proc 5][Train](10000/100000) average neg_loss: 0.34019222265481946
[proc 7][Train](10000/100000) average neg_loss: 0.3387618803232908
[proc 1][Train](10000/100000) average pos_loss: 0.18611940859258175
[proc 0][Train](10000/100000) average pos_loss: 0.1863596855700016
[proc 3][Train](10000/100000) average neg_loss: 0.33783415557444096
[proc 2][Train](10000/100000) average loss: 0.2624573111087084
[proc 6][Train](10000/100000) average neg_loss: 0.3389372945427895
[proc 5][Train](10000/100000) average loss: 0.263440335705

[proc 0][Train](13000/100000) average pos_loss: 0.18640460769832135
[proc 0][Train](13000/100000) average neg_loss: 0.33963911993801593
[proc 4][Train](13000/100000) average pos_loss: 0.1851529169678688
[proc 6][Train](13000/100000) average pos_loss: 0.18486008322238923
[proc 0][Train](13000/100000) average loss: 0.26302186350524426
[proc 2][Train](13000/100000) average pos_loss: 0.18593136365711688
[proc 3][Train](13000/100000) average pos_loss: 0.18368432275950908
[proc 4][Train](13000/100000) average neg_loss: 0.3407114086151123
[proc 1][Train](13000/100000) average pos_loss: 0.186354416847229
[proc 5][Train](13000/100000) average pos_loss: 0.1874673893749714
[proc 6][Train](13000/100000) average neg_loss: 0.3403276725709438
[proc 7][Train](13000/100000) average pos_loss: 0.18684271359443663
[proc 0][Train](13000/100000) average regularization: 0.04321873064339161
[proc 2][Train](13000/100000) average neg_loss: 0.3400227712094784
[proc 4][Train](13000/100000) average loss: 0.2629321

...

[proc 3][Train](88000/100000) average pos_loss: 0.18623369398713113
[proc 5][Train](88000/100000) average pos_loss: 0.18945906014740468
[proc 0][Train](88000/100000) average pos_loss: 0.18751904614269732
[proc 3][Train](88000/100000) average neg_loss: 0.33922946597635745
[proc 6][Train](88000/100000) average pos_loss: 0.1861594127714634
[proc 1][Train](88000/100000) average pos_loss: 0.18823234841227532
[proc 3][Train](88000/100000) average loss: 0.26273158013820647
[proc 2][Train](88000/100000) average pos_loss: 0.18581205135583878
[proc 0][Train](88000/100000) average neg_loss: 0.3417802293151617
[proc 5][Train](88000/100000) average neg_loss: 0.34401800018548967
[proc 7][Train](88000/100000) average pos_loss: 0.18624953027069568
[proc 6][Train](88000/100000) average neg_loss: 0.34089904043078423
[proc 1][Train](88000/100000) average neg_loss: 0.3431579533368349
[proc 3][Train](88000/100000) average regularization: 0.042447278913110495
[proc 4][Train](88000/100000) average pos_loss: 

[proc 2][Train](91000/100000) average pos_loss: 0.18719951257109643
[proc 7][Train](91000/100000) average pos_loss: 0.18726008512079717
[proc 5][Train](91000/100000) average pos_loss: 0.18920949061214923
[proc 2][Train](91000/100000) average neg_loss: 0.3414943594634533
[proc 7][Train](91000/100000) average neg_loss: 0.3412853932529688
[proc 6][Train](91000/100000) average pos_loss: 0.18656969740986823
[proc 4][Train](91000/100000) average pos_loss: 0.18589601247012616
[proc 3][Train](91000/100000) average pos_loss: 0.18569258235394956
[proc 2][Train](91000/100000) average loss: 0.2643469359576702
[proc 5][Train](91000/100000) average neg_loss: 0.3435679910928011
[proc 0][Train](91000/100000) average pos_loss: 0.18720377500355243
[proc 1][Train](91000/100000) average pos_loss: 0.1875340671390295
[proc 7][Train](91000/100000) average loss: 0.26427273894846437
[proc 6][Train](91000/100000) average neg_loss: 0.340806207254529
[proc 4][Train](91000/100000) average neg_loss: 0.3421996166706

[proc 1][Train](94000/100000) average pos_loss: 0.1889117314070463
[proc 7][Train](94000/100000) average pos_loss: 0.1861717364192009
[proc 2][Train](94000/100000) average pos_loss: 0.18679379215836525
[proc 4][Train](94000/100000) average pos_loss: 0.18594211426377297
[proc 1][Train](94000/100000) average neg_loss: 0.342441589564085
[proc 0][Train](94000/100000) average pos_loss: 0.18783231802284717
[proc 6][Train](94000/100000) average pos_loss: 0.18709223337471484
[proc 7][Train](94000/100000) average neg_loss: 0.3413514041006565
[proc 2][Train](94000/100000) average neg_loss: 0.3415972845554352
[proc 3][Train](94000/100000) average pos_loss: 0.18586182717978955
[proc 1][Train](94000/100000) average loss: 0.26567666056752204
[proc 4][Train](94000/100000) average neg_loss: 0.3420760493576527
[proc 5][Train](94000/100000) average pos_loss: 0.18823093013465406
[proc 6][Train](94000/100000) average neg_loss: 0.34191449159383774
[proc 0][Train](94000/100000) average neg_loss: 0.342220903

[proc 1][Train](97000/100000) average pos_loss: 0.1880246089398861
[proc 4][Train](97000/100000) average pos_loss: 0.18645547910034657
[proc 1][Train](97000/100000) average neg_loss: 0.3432003396749497
[proc 0][Train](97000/100000) average pos_loss: 0.18812814581394197
[proc 4][Train](97000/100000) average neg_loss: 0.34210098727047444
[proc 1][Train](97000/100000) average loss: 0.26561247403919697
[proc 7][Train](97000/100000) average pos_loss: 0.18720250982046127
[proc 0][Train](97000/100000) average neg_loss: 0.34284205093979836
[proc 3][Train](97000/100000) average pos_loss: 0.18487849628925324
[proc 5][Train](97000/100000) average pos_loss: 0.1895114850997925
[proc 2][Train](97000/100000) average pos_loss: 0.186315269485116
[proc 4][Train](97000/100000) average loss: 0.2642782333046198
[proc 1][Train](97000/100000) average regularization: 0.042234572384506466
[proc 6][Train](97000/100000) average pos_loss: 0.18586628663539886
[proc 0][Train](97000/100000) average loss: 0.265485098

[proc 2][Train](100000/100000) average pos_loss: 0.18659216845035553
[proc 5][Train](100000/100000) average pos_loss: 0.18918292494118213
[proc 4][Train](100000/100000) average pos_loss: 0.18557590989768505
[proc 7][Train](100000/100000) average pos_loss: 0.18690222263336181
[proc 2][Train](100000/100000) average neg_loss: 0.34122528751194475
[proc 6][Train](100000/100000) average pos_loss: 0.18689493143558503
[proc 3][Train](100000/100000) average pos_loss: 0.18645194701850415
[proc 1][Train](100000/100000) average pos_loss: 0.18814067722856997
[proc 0][Train](100000/100000) average pos_loss: 0.1877853106558323
[proc 5][Train](100000/100000) average neg_loss: 0.34356718717515466
[proc 4][Train](100000/100000) average neg_loss: 0.34288142554461953
[proc 7][Train](100000/100000) average neg_loss: 0.34167231552302835
[proc 2][Train](100000/100000) average loss: 0.26390872824192046
[proc 6][Train](100000/100000) average neg_loss: 0.3413861958682537
[proc 3][Train](100000/100000) average n

## Get Entity and Relation Embeddings
The resulting model, i.e., the entity and relation embeddings can be found under ./ckpts. (Please refer to the first line of the training log for the specific location.)

The overall process will generate 4 important files:

  - Entity embedding: ./ckpts/<model\_name>_<dataset\_name>_<run_\id>/xxx\_entity.npy
  - Relation embedding: ./ckpts/<model\_name>_<dataset\_name>_<run\_id>/xxx\_relation.npy
  - The entity id mapping, formated in <entity\_name> <entity\_id> pair: <data\_path>/entities.tsv
  - The relation id mapping, formated in <relation\_name> <relation\_id> pair: <data\_path>/relations.tsv

In [ ]:
!ls ./ckpts/TransE_l2_DRKG_0/
!ls ./train/

## A Glance of the Entity and Relation Embeddings

In [ ]:
node_emb = np.load('./ckpts/TransE_l2_DRKG_1/DRKG_TransE_l2_entity.npy')
relation_emb = np.load('./ckpts/TransE_l2_DRKG_1/DRKG_TransE_l2_relation.npy')

print(node_emb.shape)
print(relation_emb.shape)